<a href="https://colab.research.google.com/github/microprediction/AOLForTimeSeries/blob/main/mean_girl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%env API_BASE_URL=https://api.hub.crunchdao.io

env: API_BASE_URL=https://api.hub.crunchdao.io


# Momentum Attacker

This notebook demonstrates how to create an `Attacker` described in [attacker.md](https://github.com/microprediction/endersgame/blob/main/endersgame/attackers/attacker.md). You may want to glance at this [notebook](../mean_reversion_attacker/mean_reversion_attacker.ipynb) also, if you seek more context or wish to know how these attackers can be used in a new tournament.



## Setup

In [17]:
!pip install --upgrade endersgame
!pip install --upgrade pandas_ta
!pip install river

In [4]:
!pip install crunch-cli==4.0.0b6
!crunch setup --notebook enders mean-girl --token sd0tnjuKPzDFcPe7seGU09FQ8h72TVLulOaNFxEnOH8jbENE1LumhCFtECWXZLeA

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for gitignorefile: filename=gitignorefile-1.1.2-py3-none-any.whl size=6671 sha256=f369924ff4d3166ca507073e2cee55405f1d9503b5f0bfb2fedf5e10e37ec22e
  Stored in directory: /root/.cache/pip/wheels/6d/dc/c7/5875272151d55a37a44c163d1bddb1cdfff35278b82dcf4ba7
Successfully built gitignorefile
you appear to have never submitted code before
download data/X_train.parquet from https:crunch

## Imports

In [20]:
import math
import typing
import pandas as pd
from endersgame import HORIZON, Attacker, FEWMean, FEWVar
from endersgame.accounting.pnlutil import add_pnl_summaries, zero_pnl_summary
from tqdm.auto import tqdm
from endersgame import Attacker, HORIZON, EPSILON
from river import linear_model
from collections import deque
from endersgame import stream_generator_generator
from pprint import pprint
import pandas as pd
from endersgame.accounting.pnlutil import zero_pnl_summary, add_pnl_summaries

In [21]:
import crunch

crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>


# Momentum Signal

In [118]:
class MomentumSignal(Attacker):

     def __init__(self, fast_fading_factor:dict=0.1, slow_fading_factor=0.01, diff_fading_factor=0.001, threshold = 2, burn_in=100, **kwargs):
         super().__init__(**kwargs)
         self.threshold = threshold
         self.fast_ewa = FEWMean(fading_factor=fast_fading_factor)   # <--- Track fast expon-weighted moving average
         self.slow_ewa = FEWMean(fading_factor=slow_fading_factor)   # <--- Track slow expon-weighted moving average
         self.diff_var = FEWVar(fading_factor=diff_fading_factor)    # <--- Tracks mean and var of the difference between the two
         self.countdown = burn_in

     def tick(self, x:float):
         self.fast_ewa.tick(x=x)    # <--- Update the fast expon avg
         self.slow_ewa.tick(x=x)    # <--- Update the slow expon avg
         fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
         self.diff_var.tick(x=fast_minus_slow)  # <--- Update var of diff
         self.countdown -= 1        # <--- Soon we'll be warm

     def predict(self, horizon: int = None) -> float:
         """
               We buy if signal > threshold*(trailing std of signal)
         """
         if self.countdown > 0:
             return 0    # Not warmed up
         fast_minus_slow = self.fast_ewa.get() - self.slow_ewa.get()
         try:
             fast_minus_slow_std = math.sqrt( self.diff_var.get())
             decision = int(fast_minus_slow/(self.threshold*fast_minus_slow_std))  # <--- Create a buy (>0) or sell (<0) decision
             return decision
         except ArithmeticError:
             return 0



## Run the attacker on real data

We reset the attacker every time it encounters a new stream, but track aggregate statistics.

In [89]:
x_train, x_test = crunch.load_streams()

download data/X_train.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/X_train.parquet (2248900 bytes)
already exists: file length match
download data/y_train.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/y_train.parquet (1828809 bytes)
already exists: file length match
download data/X_test.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/X_test_reduced.parquet (141137 bytes)
already exists: file length match
download data/y_test.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/y_test_reduced.parquet (145664 bytes)
already exists: file length match
download data/example_prediction.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/example_prediction_reduced.parquet (145664 bytes)
already exists: file length match


In [130]:
total_pnl = []

max_streams = 1000
stream_count = 0
C = 0.001
eps = 0.01
threshold = 3
mode = 1
max_history_len = 40

for stream in tqdm(x_train):
    stream_count += 1
    #momentum_signal = MomentumSignal(fast_fading_factor=0.1, slow_fading_factor=0.01, diff_fading_factor=0.001, threshold = 2, burn_in=100)
    #uncalibrated = UncalibratedAttacker(max_history_len=max_history_len, C=C, eps=eps, mode=mode, threshold=threshold)
    #attacker = CalibratedAttacker(attacker=uncalibrated, fading_factor=0.001)
    attacker = MomentumSignal(fast_fading_factor=0.1, slow_fading_factor=0.001, diff_fading_factor=0.001, threshold = 1, burn_in=100)
    pnl = zero_pnl_summary()

    for message in tqdm(stream, leave=False):
        if message.get('x') is None:
            raise ValueError('huh')
        attacker.tick_and_predict(x=message['x'])

    stream_pnl = attacker.pnl.summary()

    pnl = add_pnl_summaries(pnl, stream_pnl)
    try:
      pnl.update({
          'profit_per_decision': pnl['total_profit'] / pnl['num_resolved_decisions']
      })
    except ArithmeticError:
      pass

    total_pnl.append(pnl)
    if stream_count>=max_streams:
       break

total_pnl = pandas.DataFrame(total_pnl)
total_pnl

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/4892 [00:00<?, ?it/s]

  0%|          | 0/18886 [00:00<?, ?it/s]

  0%|          | 0/14472 [00:00<?, ?it/s]

  0%|          | 0/4396 [00:00<?, ?it/s]

...

  0%|          | 0/4299 [00:00<?, ?it/s]

  0%|          | 0/17312 [00:00<?, ?it/s]

  0%|          | 0/13501 [00:00<?, ?it/s]

  0%|          | 0/3874 [00:00<?, ?it/s]

,total_profit,num_resolved_decisions,wins,losses,current_ndx,profit_per_decision
0,-116.777619,2123,963,1160,4892,-0.055006
1,-468.513810,8799,4070,4729,18886,-0.053246
2,139.451429,6962,3397,3565,14472,0.020030
3,43.566667,2010,961,1049,4396,0.021675
4,-96.710000,2831,1355,1476,4777,-0.034161
...,...,...,...,...,...,...
75,-72.855000,2413,1135,1278,3585,-0.030193
76,26.071163,3203,1411,1792,4299,0.008140
77,168.994148,9241,4618,4623,17312,0.018287
78,73.792570,7917,3906,4011,13501,0.009321


In [132]:
total_pnl.sum()

,0
total_profit,271.974797
num_resolved_decisions,430482.000000
wins,212462.000000
losses,218020.000000
current_ndx,801445.000000
profit_per_decision,-0.769202


## CrunchDAO Code Interface

[Submitting to the CrunchDAO platform requires 2 functions, `train` and `infer`.](https://docs.crunchdao.com/competitions/code-interface) Any line that is not in a function or is not an import will be commented when the notebook is processed.

The content of the function is the same as the example, but the train must save the model to be read in infer.

In [13]:
def train():
    pass  # no train

In [133]:
def infer(
    stream: typing.Iterator[dict],
):
    max_streams = 1
    C = 0.001
    eps = 0.01
    threshold = 3
    mode = 1
    max_history_len = 40

    #uncalibrated = UncalibratedAttacker(max_history_len=max_history_len, C=C, eps=eps, mode=mode, threshold=threshold)
    #attacker = MomentumSignal(fast_fading_factor=0.025, slow_fading_factor=0.1, diff_fading_factor=0.001, threshold = 2, burn_in=100)
    #attacker = CalibratedAttacker(attacker=uncalibrated, fading_factor=0.001)
    attacker = MomentumSignal(fast_fading_factor=0.1, slow_fading_factor=0.001, diff_fading_factor=0.001, threshold = 1, burn_in=100)

    total_pnl = zero_pnl_summary()

    yield  # mark as ready

    for message in stream:
        yield attacker.tick_and_predict(x=message['x'])

    stream_pnl = attacker.pnl.summary()
    total_pnl = add_pnl_summaries(total_pnl, stream_pnl)
    try:
      total_pnl.update({
          'profit_per_decision': total_pnl['total_profit'] / total_pnl['num_resolved_decisions']
      })
    except ArithmeticError:
       pass

    print(total_pnl)

In [134]:
crunch.test()

print("Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/endersgame/submit/via/notebook")

ignoring cell #45: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 26)
ignoring cell #78: expected ':' (<unknown>, line 17)
ignoring cell #92: unindent does not match any outer indentation level (<unknown>, line 32)
ignoring cell #109: invalid syntax (<unknown>, line 11)


00:33:13 forbidden library: pandas_ta
00:33:13 
00:33:13 started
00:33:13 running local test
00:33:13 internet access isn't restricted, no check will be done
00:33:13 
00:33:14 starting stream loop...


download data/X_train.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/X_train.parquet (2248900 bytes)
already exists: file length match
download data/y_train.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/y_train.parquet (1828809 bytes)
already exists: file length match
download data/X_test.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/X_test_reduced.parquet (141137 bytes)
already exists: file length match
download data/y_test.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/y_test_reduced.parquet (145664 bytes)
already exists: file length match
download data/example_prediction.parquet from https:crunchdao--competition--staging.s3.eu-west-1.amazonaws.com/data-releases/72/example_prediction_reduced.parquet (145664 bytes)
already exists: file length match


00:33:15 call: train - stream.len=80
00:33:15 looping stream=`aud-jpy` (1/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`aud-nzd` (2/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`aud-usd` (3/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`eur-aud` (4/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`eur-chf` (5/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`eur-gbp` (6/20)
00:33:15 call: infer (1/1)
00:33:15 looping stream=`eur-jpy` (7/20)
00:33:15 call: infer (1/1)


{'total_profit': -112.96000000002896, 'num_resolved_decisions': 1266, 'wins': 566, 'losses': 700, 'current_ndx': 1987, 'profit_per_decision': -0.08922590837285067}
{'total_profit': -16.517142857146958, 'num_resolved_decisions': 1191, 'wins': 584, 'losses': 607, 'current_ndx': 1859, 'profit_per_decision': -0.013868297948905926}
{'total_profit': 252.7928571427866, 'num_resolved_decisions': 1335, 'wins': 706, 'losses': 629, 'current_ndx': 1647, 'profit_per_decision': 0.18935794542530832}
{'total_profit': 370.8005263159947, 'num_resolved_decisions': 1491, 'wins': 750, 'losses': 741, 'current_ndx': 1967, 'profit_per_decision': 0.24869250591280664}
{'total_profit': 48.50000000000255, 'num_resolved_decisions': 750, 'wins': 400, 'losses': 350, 'current_ndx': 1839, 'profit_per_decision': 0.06466666666667006}
{'total_profit': 816.7833333324924, 'num_resolved_decisions': 1380, 'wins': 816, 'losses': 564, 'current_ndx': 1560, 'profit_per_decision': 0.5918719806757191}


...

{'total_profit': 16.08304347826083, 'num_resolved_decisions': 1383, 'wins': 670, 'losses': 713, 'current_ndx': 2023, 'profit_per_decision': 0.011629098682762712}
{'total_profit': -73.27333333329324, 'num_resolved_decisions': 1044, 'wins': 512, 'losses': 532, 'current_ndx': 1660, 'profit_per_decision': -0.07018518518514678}
{'total_profit': 335.9200000000546, 'num_resolved_decisions': 1008, 'wins': 547, 'losses': 461, 'current_ndx': 1634, 'profit_per_decision': 0.3332539682540224}
{'total_profit': -76.02333333334555, 'num_resolved_decisions': 969, 'wins': 520, 'losses': 449, 'current_ndx': 1782, 'profit_per_decision': -0.07845545235639376}
{'total_profit': -375.19333333372657, 'num_resolved_decisions': 1056, 'wins': 473, 'losses': 583, 'current_ndx': 1949, 'profit_per_decision': -0.35529671717208955}
{'total_profit': 214.23007334963796, 'num_resolved_decisions': 1630, 'wins': 953, 'losses': 677, 'current_ndx': 1917, 'profit_per_decision': 0.13142949285253863}


00:33:15 looping stream=`usd-zar` (20/20)
00:33:15 call: infer (1/1)
00:33:15 save prediction - path=data/prediction.csv


{'total_profit': 154.58000000004836, 'num_resolved_decisions': 922, 'wins': 544, 'losses': 378, 'current_ndx': 1398, 'profit_per_decision': 0.16765726681133228}
{'total_profit': -36.94648648648759, 'num_resolved_decisions': 796, 'wins': 388, 'losses': 408, 'current_ndx': 1659, 'profit_per_decision': -0.04641518402825075}


00:33:16 prediction is valid
00:33:16 ended
00:33:16 duration - time=00:00:03
00:33:16 memory - before="506.49 MB" after="539.67 MB" consumed="33.18 MB"


Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/endersgame/submit/via/notebook
